In [1]:
import pandas as pd
import geopandas as gpd
from shapely import wkt
import h3
from shapely.geometry import Polygon

<div>
    <span style ="font-size: 40px; font-weight: bold; color: #8EB944">
        Load the data
    </span>
    
<hr style="color: #8EB944; height: 3px;background-color: #8EB944;border: none">
</div>

In [2]:
# load the census tract areas as a geodataframe
census_gdf = gpd.read_file('Boundaries.geojson', crs='epsg:4326')

# load the poi data as a geodataframe
poi_gdf = gpd.read_file('POI_data.geojson', crs='epsg:4326')

# load the landuse data as a geodataframe
landuse_gdf = gpd.read_file('landuse_chicago.geojson',  crs='epsg:4326')

# load the taxi trips as a dataframe (only 500k rides are needed for this notebook)
taxi_df = pd.read_csv('clean_taxi_data.csv', nrows=500000)



<div>
    <span style ="font-size: 30px; font-weight: bold; color: #8EB944">
        Formatting
    </span>
    
<hr style="color: #8EB944; height: 3px;background-color: #8EB944;border: none">
</div>


<span style ="font-size: 18px; font-weight: bold;color: #43556A;">
        1. Drop rides with no dropoff location
</span>
<hr style="color: #8EB944; height: 1px;background-color: #43556A;border: none">

In [3]:
# for the clustering only trips i.e. start and dropoff can be used
taxi_df.drop(taxi_df[taxi_df.dropoff_location.isna()].index, inplace = True)

<span style ="font-size: 18px; font-weight: bold;color: #43556A;">
        2. Recover the formatting of some variables
</span>
<hr style="color: #8EB944; height: 1px;background-color: #43556A;border: none">


In [4]:
# drop the 'Unnamed: 0' column
taxi_df.drop(columns=['Unnamed: 0'],inplace = True)

#  convert the strings to point objects
taxi_df['pickup_location'] = taxi_df['pickup_location'].apply(wkt.loads)
taxi_df['dropoff_location'] = taxi_df['dropoff_location'].apply(wkt.loads)

<div>
    <span style ="font-size: 30px; font-weight: bold; color: #8EB944">
        Write helper functions
    </span>
    
<hr style="color: #8EB944; height: 3px;background-color: #8EB944;border: none">
</div>

Write a function that gets all the unique pairs of census tract and location.

In [5]:
def get_unique_census_location_pairs(taxi_df):
    # Extract unique pair of pickup census tract and pickup location
    unique_pickup = taxi_df[['pickup_census', 'pickup_location']].drop_duplicates()
    
    # Extract unique pair of dropoff census tract and dropoff location
    unique_dropoff = taxi_df[['dropoff_census', 'dropoff_location']].drop_duplicates()
    
    # Rename columns to make them consistent
    unique_pickup.columns = ['census', 'location']
    unique_dropoff.columns = ['census', 'location']
    
    # Combine both dataframes and drop any duplicates
    unique_loc = pd.concat([unique_pickup, unique_dropoff]).drop_duplicates()
    
    # Reset index
    unique_loc.reset_index(drop=True, inplace=True)
    
    return unique_loc

Write a function that initializes an empty feature map that contains the hexagon id for the specified  resolution, the polygon of the hexagon and the centroid of the hexagon as well as the census tract id.

In [6]:
def process_locations_with_h3(unique_loc, resolution=7):
    
    # Calculate the h3 hexagon ID for each location using the specified resolution
    unique_loc["h3_hex_id"] = unique_loc.apply(
        lambda row: h3.geo_to_h3(row['location'].y, row['location'].x, resolution) 
        if row['location'] 
        else 0,
        axis=1
    )

    # Drop duplicates based on the h3 hexagon ID
    df_unique_hex = unique_loc.drop_duplicates(subset="h3_hex_id")[['census', 'h3_hex_id']]

    # Calculate the geometry of the hexagons
    df_unique_hex['geometry_hex'] = df_unique_hex.apply(
        lambda x: Polygon(h3.h3_to_geo_boundary(x["h3_hex_id"], geo_json=True)), 
        axis=1
    )

    # Calculate the centroid of each hexagon
    df_unique_hex['centroid'] = df_unique_hex['geometry_hex'].apply(lambda x: x.centroid)

    # Create a GeoDataFrame with the centroids as geometry (change the crs of the centroids)
    feature_map = gpd.GeoDataFrame(df_unique_hex, geometry='centroid', crs='epsg:4326').to_crs(epsg=26971).reset_index()

    return feature_map

Write a function that calculate the distance to the closest airport for the centroid of each hexagon.

In [7]:
def calculate_min_distance_to_airport(feature_map, poi_gdf):
    
     # Check if the active geometry is centroid and if the CRS is 26971
    if feature_map.geometry.name != 'centroid':
        raise ValueError("The active geometry of feature_map is not 'centroid'.")
    if feature_map.crs.to_epsg() != 26971:
        raise ValueError("The CRS of feature_map is not EPSG:26971.")
        
    # Project the locations of the airports to a CRS better fitted for calculating distances in North America
    airports = poi_gdf[poi_gdf.Category == 'Airport'].to_crs(epsg=26971)
    
    # Calculate the distance to the closest airport for each hexagon centroid
    feature_map['min_dist_airport'] = feature_map.centroid.apply(
        lambda x: airports.distance(x).round().min()
    )


Write a function that calculate the distance to the city centre for the centroid of each hexagon.

In [8]:
def calculate_distance_to_city_centre(feature_map, census_gdf, commarea='32'):
    
    # Check if the active geometry is centroid and if the CRS is 26971
    if feature_map.geometry.name != 'centroid':
        raise ValueError("The active geometry of feature_map is not 'centroid'.")
    if feature_map.crs.to_epsg() != 26971:
        raise ValueError("The CRS of feature_map is not EPSG:26971.")
    
    # Index the community area 32 to get the census tracts that lie within that area
    loop = census_gdf[census_gdf.commarea == commarea].to_crs(epsg=26971)
    
    # Calculate the distance to the city centre in meters for each hexagon centroid
    feature_map['dist_centre'] = feature_map.centroid.apply(
        lambda x: loop.distance(x).round().min()
    )


Write a function that checks whether there is one of the 2 airports in the hexagon. We place a buffer of 1000 metres around the location of the airports , because we did not get a polygon for the airports. A buffer of 1000 metres perfectly catches the 4 census tracts that have the 2 airports inside.

In [9]:
def airport_in_hex(feature_map, poi_gdf):
    # Check if the active geometry is geometry_hex and if the CRS is 4326
    if feature_map.geometry.name != 'geometry_hex':
        raise ValueError("The active geometry of feature_map is not the hexagon polygon.")
    if feature_map.crs.to_epsg() != 4326:
        raise ValueError("The CRS of feature_map is not EPSG:4326.")
    
    # Filter the POIs by the specified category
    airports = poi_gdf[poi_gdf.Category == 'Airport']
    
    # Reproject the airports to a suitable CRS for distance calculations
    airports = airports.to_crs(epsg=26971)
    
    # Apply a 1000-meter buffer around the POI locations (we only have the airport locations as a single point)
    airports_buffer = airports.copy()
    airports_buffer['geometry'] = airports_buffer.geometry.buffer(1000)
    
    # Reproject the feature_map to the same CRS
    feature_map_projected = feature_map.to_crs(epsg=26971)
    
    # Perform a spatial join to find all hexagons that intersect with the buffered airports
    airport_in_hex = gpd.sjoin(airports_buffer, feature_map_projected, how='inner', predicate='intersects')
    

    # Reproject feature_map back to the original CRS
    feature_map = feature_map_projected.to_crs(epsg=4326)
    
    # Add the count of airports to the respective hexagon in feature_map
    feature_map['airport_in_hex'] = feature_map.h3_hex_id.apply(
        lambda x: 1 if x in list(airport_in_hex.h3_hex_id) else 0.0)
    return feature_map

Write a function that counts the number of a specified poi category in the hexagon and adds it inplace to the feature map for the specific hexagon resolution.

In [10]:
def count_poi_in_hexagons(feature_map, poi_gdf, poi_category, column_name):
    
    # Check if the active geometry is geometry_hex and if the CRS is 4326
    if feature_map.geometry.name != 'geometry_hex':
        raise ValueError("The active geometry of feature_map is not the hexagon polygon.")
    if feature_map.crs.to_epsg() != 4326:
        raise ValueError("The CRS of feature_map is not EPSG:4326.")
    
    # Filter the POIs by the specified category
    pois = poi_gdf[poi_gdf.Category == poi_category]
    
    # Perform a spatial join to find all hexagons that contain the POIs
    pois_in_hex = gpd.sjoin(pois, feature_map, how='inner', predicate='within')
    
    # Count the number of POIs in each hexagon
    poi_counter = dict(pois_in_hex['h3_hex_id'].value_counts())
    
    # Add the count of POIs to the respective hexagon in feature_map
    feature_map[column_name] = feature_map.h3_hex_id.apply(
        lambda x: poi_counter[x] if x in poi_counter else 0
    )

Write a function that calculates the hexagon id for the census tracts for the specified resolution, then calculates the landusage in percentage for all categories for each hexagon id. The output looks like this {'hex_id1': {'COMMERCIAL': 1.32, 'COMMUNICATION/UTILITIES': 20.84....},
 'hex_id2': {'COMMERCIAL': 5.49,'NON-PARCEL AREAS': 66.27...}.....}.

In [11]:
def create_landuse_hex_dict(unique_loc, landuse, resolution=7):

    # change the crs of the landuse data such that it matches the other data
    landuse.to_crs(epsg=26971, inplace=True)
    
    # Calculate the h3 hexagon ID for each location using the specified resolution
    unique_loc["h3_hex_id"] = unique_loc.apply(
        lambda row: h3.geo_to_h3(row['location'].y, row['location'].x, resolution) 
        if row['location'] 
        else 0,
        axis=1
    )

    # Drop duplicates based on the h3 hexagon ID
    df_unique_hex = unique_loc.drop_duplicates(subset="h3_hex_id")[['census', 'h3_hex_id']]

    # create a dictionary that maps the census tract id to the hexagon id
    census_hex_dict = df_unique_hex.set_index('census')['h3_hex_id'].to_dict()

    # get the hexagon id for each parcel in the landuse geodataframe
    landuse['hex_id'] = landuse.geoid10.apply(lambda x: census_hex_dict[int(x)] \
                                                 if int(x) in census_hex_dict
                                                 else None)
    # calculate the area for each parcel
    landuse['area'] = landuse['geometry'].area
    
    # Group by hexagon id and land use category to sum the areas (also reset the indexes)
    landuse_by_category = landuse.groupby(['hex_id', 'LANDUSE'])['area'].sum().reset_index()

    # Calculate the total area for each census tract
    total_area = landuse_by_category.groupby('hex_id')['area'].sum().reset_index()

    # rename the variable
    total_area = total_area.rename(columns={'area': 'total_area'})

    # Merge total_area with landuse_by_category and calculate the percentage
    landuse_by_category = landuse_by_category.merge(total_area, on='hex_id')

    # divide the area for each parcel by the area of the hexagon is is within
    landuse_by_category['percentage'] = ((landuse_by_category['area'] / landuse_by_category['total_area'])*100).round(2)

    # Create a dictionary to store results
    landuse_dict = {}

    # Populate the dictionary with the data
    for _, row in landuse_by_category.iterrows():
        hex_id = row['hex_id']
        landuse = row['LANDUSE']
        percentage = row['percentage']
    
        if hex_id not in landuse_dict:
            landuse_dict[hex_id] = {}
    
        landuse_dict[hex_id][landuse] = percentage

    return landuse_dict

Write a function that adds the landusage for a landuse category for each hexagon id to the feature map

In [12]:
def add_landuse_percentage(feature_map, landuse_dict, landuse_type, column_name):
    
    # Add the percentage of the specified land use type to the feature_map
    feature_map[column_name] = feature_map.h3_hex_id.apply(
        lambda x: landuse_dict[x][landuse_type] 
        if landuse_type in landuse_dict.get(x, {}) 
        else 0.0
    )

<div>
    <span style ="font-size: 30px; font-weight: bold; color: #8EB944">
        Create Feature Maps
    </span>
    
<hr style="color: #8EB944; height: 3px;background-color: #8EB944;border: none">
</div>

For the Support Vector Machines and the Neural Network, we will be using hexagon resolution 6 and 7. Hence, we create a feature map for each resolution

In [13]:
hexagon_resolutions = [6,7]
feature_maps = []

for resolution in hexagon_resolutions:
    #1. step: get unique locations 
    unique_loc = get_unique_census_location_pairs(taxi_df)

    # 2. initialize the feature map for the hexagon resolution of 7
    feature_map = process_locations_with_h3(unique_loc, resolution=resolution)

    # 3. add distance to closest airport
    calculate_min_distance_to_airport(feature_map, poi_gdf)
    
    # 4. add distance to the city centre
    calculate_distance_to_city_centre(feature_map, census_gdf, commarea='32')

    # 5. Add the occurences of a POI category

    # we must change the geometry of the feature_map to the polygon for the spatial join
    feature_map.set_geometry('geometry_hex', inplace=True)
    feature_map.set_crs('EPSG:4326', inplace=True) 

    # number of stadiums in hexagon
    count_poi_in_hexagons(feature_map, poi_gdf, 'Stadium', 'num_stadiums')
    # number of hotels in hexagon
    count_poi_in_hexagons(feature_map, poi_gdf, 'Hotel', 'num_hotels')
    # number of bars/clubs in hexagon
    count_poi_in_hexagons(feature_map, poi_gdf, 'Bar/ Night Club', 'num_bars')

    # airport in hexagon
    feature_map = airport_in_hex(feature_map, poi_gdf)
    
    #1. step: get unique locations 
    unique_loc = get_unique_census_location_pairs(taxi_df)

    #2. step: create landuse dictionary for specified hexagon resolution
    landuse = create_landuse_hex_dict(unique_loc, landuse_gdf, resolution=resolution)

    #3. step: add the landuse percentage for each hexagon
    add_landuse_percentage(feature_map, landuse, 'TRANSPORTATION', 'perc_transport')
    add_landuse_percentage(feature_map, landuse, 'RESIDENTIAL', 'perc_resid')
    add_landuse_percentage(feature_map, landuse, 'COMMERCIAL', 'perc_commerc')
    add_landuse_percentage(feature_map, landuse, 'OPEN SPACE', 'perc_open')
    
    feature_maps.append(feature_map)

In [14]:
feature_maps[0]['centroid'] = feature_maps[0]['centroid'].astype(str)
feature_maps[1]['centroid'] = feature_maps[1]['centroid'].astype(str)

In [20]:
feature_maps[0].to_file('spatial_features_hex6.geojson')
feature_maps[1].to_file('spatial_features_hex7.geojson')